In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras import metrics
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from sklearn.externals import joblib
import timeit

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


ref: https://towardsdatascience.com/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-i-hyper-parameter-8129009f131b
ref2: https://github.com/maxpumperla/hyperas

In [2]:
# load training, validation and test sets

def data_from_previous_notebook():
    X_train, X_val, X_test, y_train, y_val, y_test = joblib.load('X_y.pkl')
    return X_train, y_train, X_val, y_val

In [3]:
# check and see if the datasets are loaded correctly
# X_train.shape

In [4]:
# follow the blog post above to set up the architecture for tuning neural net

def nn_model(X_train, y_train, X_val, y_val):
    
    model = Sequential()
    model.add(Dense({{choice([128, 256, 512, 1024])}}, input_shape=(104,))) # change input size to 104 for my use case
    model.add(Activation({{choice(['relu', 'sigmoid', 'tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([128, 256, 512, 1024])}}))
    model.add(Activation({{choice(['relu', 'sigmoid', 'tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    
    if conditional({{choice(['two', 'ten'])}}) == 'ten':
        # 3rd layer
        model.add(Dense({{choice([128, 256, 512, 1024])}}))
        model.add(Activation({{choice(['relu', 'sigmoid', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
        
        # 4th layer
        model.add(Dense({{choice([128, 256, 512, 1024])}}))
        model.add(Activation({{choice(['relu', 'sigmoid', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
        
        # 5th layer
        model.add(Dense({{choice([128, 256, 512, 1024])}}))
        model.add(Activation({{choice(['relu', 'sigmoid', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
        
        # 6th layer
        model.add(Dense({{choice([128, 256, 512, 1024])}}))
        model.add(Activation({{choice(['relu', 'sigmoid', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
        
        # 7th layer
        model.add(Dense({{choice([128, 256, 512, 1024])}}))
        model.add(Activation({{choice(['relu', 'sigmoid', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
        
        # 8th layer
        model.add(Dense({{choice([128, 256, 512, 1024])}}))
        model.add(Activation({{choice(['relu', 'sigmoid', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
        
        # 9th layer
        model.add(Dense({{choice([128, 256, 512, 1024])}}))
        model.add(Activation({{choice(['relu', 'sigmoid', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
        
        # 10th layer
        model.add(Dense({{choice([128, 256, 512, 1024])}}))
        model.add(Activation({{choice(['relu', 'sigmoid', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))

        
    model.add(Dense(2)) # in my use case, it's binary so I change the number of neurons to 2
    model.add(Activation('softmax'))
    adam = keras.optimizers.Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
    rmsprop = keras.optimizers.RMSprop(lr={{choice([10**-3, 10**-2, 10**-1])}})
    sgd = keras.optimizers.SGD(lr={{choice([10**-3, 10**-2, 10**-1])}})
   
    choiceval = {{choice(['adam', 'sgd', 'rmsprop'])}}
    if choiceval == 'adam':
        optim = adam
    elif choiceval == 'rmsprop':
        optim = rmsprop
    else:
        optim = sgd
        
    # use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optim, metrics=[metrics.categorical_accuracy])
    model.fit(X_train, y_train,
              batch_size={{choice([128,256,512])}},
              nb_epoch=20,
              verbose=2,
              validation_data=(X_val, y_val))
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [5]:
# run the optimizer
clock_in=timeit.default_timer()

best_run, best_model = optim.minimize(model=nn_model,
                                      data=data_from_previous_notebook,
                                      algo=tpe.suggest,
                                      max_evals=30,
                                      trials=Trials(),
                                      notebook_name='neural_net')

clock_out=timeit.default_timer()
print("time taken to run hyperopt-sklearn optimizer:", clock_out-clock_in)

>>> Imports:
#coding=utf-8

try:
    import keras
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Dropout, Activation
except:
    pass

try:
    from keras.optimizers import SGD, Adam, RMSprop
except:
    pass

try:
    from keras import metrics
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from sklearn.externals import joblib
except:
    pass

try:
    import timeit
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [128, 256, 512, 1024]),
        'Activation': hp.choice('Activation', ['relu', 'sigmoid', 'tanh']),
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dense_1': hp.choice('Dense_1', [128, 256, 512, 1024]),
        'Activation_1': hp.choice('Activation_1', [

/Users/peterlin/donor_classifier/src/modeling/temp_model.py:131: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 24129 samples, validate on 6033 samples
Epoch 1/20
 - 1s - loss: 0.9946 - categorical_accuracy: 0.7783 - val_loss: 1.4417 - val_categorical_accuracy: 1.0000
Epoch 2/20
 - 0s - loss: 1.1511 - categorical_accuracy: 0.7512 - val_loss: 0.9630 - val_categorical_accuracy: 1.0000
Epoch 3/20
 - 0s - loss: 1.2548 - categorical_accuracy: 0.7365 - val_loss: 1.5173 - val_categorical_accuracy: 0.0000e+00
Epoch 4/20
 - 0s - loss: 1.2555 - categorical_accuracy: 0.7427 - val_loss: 1.0535 - val_categorical_accuracy: 1.0000
Epoch 5/20
 - 0s - loss: 1.2163 - categorical_accuracy: 0.7231 - val_loss: 1.5249 - val_categorical_accuracy: 1.0000
Epoch 6/20
 - 0s - loss: 1.2512 - categorical_accuracy: 0.7366 - val_loss: 1.9395 - val_categorical_accuracy: 1.0000
Epoch 7/20
 - 0s - loss: 1.1393 - categorical_accuracy: 0.7736 - val_loss: 1.4856 - val_categorical_accuracy: 1.0000
Epoch 8/20
 - 0s - loss: 1.2170 - categorical_accuracy: 0.7748 - val_loss: 1.1283 - val_categorical_accuracy: 1.0000
Epoch 9/20


In [6]:
# now check for the best run
print("best run", best_run)

best run {'Activation': 0, 'Activation_1': 1, 'Activation_2': 0, 'Activation_3': 1, 'Activation_4': 2, 'Activation_5': 2, 'Activation_6': 1, 'Activation_7': 1, 'Activation_8': 2, 'Activation_9': 0, 'Dense': 1, 'Dense_1': 1, 'Dense_2': 2, 'Dense_3': 3, 'Dense_4': 3, 'Dense_5': 3, 'Dense_6': 0, 'Dense_7': 2, 'Dense_8': 1, 'Dense_9': 0, 'Dropout': 0.6032923044779506, 'Dropout_1': 0.26695407042103014, 'Dropout_2': 0.617666722573424, 'Dropout_3': 0.053267452425264206, 'Dropout_4': 0.5235730187653925, 'Dropout_5': 0.8444132599754368, 'Dropout_6': 0.44390441345306564, 'Dropout_7': 0.039825089762879906, 'Dropout_8': 0.11245974166556161, 'Dropout_9': 0.3207527760045966, 'batch_size': 1, 'choiceval': 1, 'conditional': 0, 'lr': 0, 'lr_1': 1, 'lr_2': 2}


Now I have the best neural net parameters to compare with random forest.

### Test Neural Net and Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
best_rf_clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.2113512273031174,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=65, n_jobs=1, oob_score=False, random_state=2,
            verbose=False, warm_start=False)

In [12]:
X_train, X_val, X_test, y_train, y_val, y_test = joblib.load('X_y.pkl')

In [13]:
# train the best random forest classifier with training and validation data
best_rf_clf.fit(X_train, y_train)
best_rf_clf.fit(X_val, y_val)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.2113512273031174,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=65, n_jobs=1, oob_score=False, random_state=2,
            verbose=False, warm_start=False)

In [22]:
best_nn_model = Sequential()
# 0
best_nn_model.add(Dense(256, input_shape=(104,), activation='relu'))
best_nn_model.add(Dropout(0.6032923044779506))
# 1
best_nn_model.add(Dense(256, activation='sigmoid'))
best_nn_model.add(Dropout(0.26695407042103014))
# output 
best_nn_model.add(Dense(2, activation='softmax'))

sgd = keras.optimizers.SGD(lr=10**-2)

best_nn_model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=[metrics.categorical_accuracy])

In [15]:
best_rf_clf.score(X=X_test, y=y_test)

0.047543160690571046